# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
!pip3 install -U random-forest-mc

In [2]:
%load_ext watermark

In [158]:
import pandas as pd
import numpy as np
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import load_file_json, dump_file_json, LoadDicts
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score
from collections import Counter
import hashlib
import joblib

In [37]:
from tqdm import tqdm

# from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display
def md(arg):
    display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [38]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.27.0

Compiler    : GCC 10.2.1 20210110
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 8
Architecture: 64bit

Git hash: 3c1a5a8fd4f67e59495a7826bcaa83ce3055e1b9

Git repo: https://github.com/ysraell/random-forest-mc.git

Git branch: main

pandas: 1.3.2
numpy : 1.20.3
joblib: 1.0.1

CPU	: Intel(R) Xeon(R) CPU E3-1241 v3 @ 3.50GHz
Mem:            31G
Swap:          4.0G


In [187]:
def dfany2dfnumeric(dfA, dfB):
    for col in dfA.columns:
        mapper = {val: i for i,val in enumerate(dfA[col].unique().tolist()+dfB[col].unique().tolist())}
        dfA[col] = dfA[col].apply(lambda x: mapper[x])
        dfB[col] = dfB[col].apply(lambda x: mapper[x])
    return dfA, dfB

# Tinanic

In [188]:
dicts = LoadDicts("../tests/")
dataset_dict = dicts.datasets_metadata
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"].replace('~','/work'))[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
target_col = params['target_col']
dataset = dataset[params['ds_cols']+[target_col]].dropna().reset_index(drop=True)
dataset[target_col] = dataset[target_col].astype(str)

In [198]:
test_size = 0.5
max_test_size = 100
max_train_size = 100
#def fullCycleAnalysis(dataset,params,test_size_frac, max_test_size, max_train_size, split_seeds, )
target_col = params['target_col']
ds_cols = params["ds_cols"]
df_classes = [dataset.query(f'{target_col} == "{target_val}"').reset_index(drop=True).copy() for target_val in dataset[target_col].unique()]

Results = []
    
# Each seed is a experiment.
for seed in split_seeds:
    
    df_train_list = []
    df_test_list = []
    while df_classes:
        df = df_classes.pop(0)
        df_train, df_test = train_test_split(df, test_size=test_size, random_state=seed)
        if df_train.shape[0] > max_train_size:
            df_train = df_train.sample(n = max_train_size, random_state=seed)
        if df_test.shape[0] > max_test_size:
            df_test = df_test.sample(n = max_test_size, random_state=seed)
        df_train_list.append(df_train.copy())
        df_test_list.append(df_test.copy())
        del df
    dataset_train = pd.concat(df_train_list).sample(dataset_train.shape[0]).reset_index(drop=True).copy()
    del df_train_list
    dataset_test = pd.concat(df_test_list).sample(dataset_test.shape[0]).reset_index(drop=True).copy()
    del df_test_list

    y_test = dataset_test[target_col].tolist()
    
    # Training step
    cls = RandomForestMC(
        n_trees=64, target_col=target_col, max_discard_trees=2,
        th_start=0.9999, batch_train_pclass = max_train_size // 2, batch_val_pclass = max_train_size // 2
    )
    cls.process_dataset(dataset_train)
    cls.fitParallel(max_workers=16, thread_parallel_method=False)
    y_pred = cls.testForest(dataset_test)
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'RFMC', F1_M, AUC_ROC_M, RECALL, TP_1))
    
    MLPC = MLPClassifier(hidden_layer_sizes=(200,), max_iter=10000)
    dataset_train_num, dataset_test_num = dfany2dfnumeric(dataset_train[ds_cols], dataset_test[ds_cols])
    # scaler = StandardScaler()
    # scaler.fit(dataset_train_num[ds_cols].to_numpy())  
    # X_train = scaler.transform(dataset_train_num[ds_cols].to_numpy())  
    # X_test = scaler.transform(dataset_test_num[ds_cols].to_numpy())  
    MLPC.fit(dataset_train_num[ds_cols].to_numpy(), dataset_train[target_col].to_numpy())
    y_pred = MLPC.predict(dataset_test_num[ds_cols].to_numpy()).tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    
    Results.append((seed, 'MLPC', F1_M, AUC_ROC_M, RECALL, TP_1))
    
    break

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/tmp/ipykernel_52/4177174917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfA[col] = dfA[col].apply(lambda x: mapper[x])
/tmp/ipykernel_52/4177174917.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [199]:
Results

[(43, 'RFMC', 0.7740453415681253, 0.7749999999999999, 0.71, 0.71),
 (43, 'MLPC', 0.7799119647859143, 0.78, 0.76, 0.76)]

# Credit Card Fraude Detection

In [6]:
df = pd.read_csv('/work/data/creditcard_trans_int.csv')
target_col = 'Class'
ds_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
df[target_col].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
len(ds_cols)

29

In [9]:
# How much for test step:
N_fraud_test = 200
N_truth_test = int(2e4)

# How much for training step:
N_truth_train = int(2e5)
# The remaing data for fraud is all used.

# How much (survived) trees:
T = 34

# Parameters for bootstrap

# Data for each tree:
N_T = int((df[target_col].value_counts()[1] - N_fraud_test)/2) # remaing data for fraud.
N_V = N_T # keep balanced amount for each classe.

# How much features:
n_F_max = len(ds_cols)
n_F_min = 20
# From EDA: decision trees with small amount of features could be not so efficient.

# Droped trees limit:
D = 256
# When a total of D trees are dropped, the validation threshold decreases delta_Th, restarting the counting.

# Delta Threshold
delta_th = 0.001
th_start = 0.999
get_best_tree = True
# The validation threshold decreases dynamically as need to selection more specialized tree.

# Metric in validation
M = 'Accuracy'
# This parameter have no effect, only to say which metric is used in validation process.

#Seeds
split_seeds = [43, 47, 53, 59]
# One for each experiment.

In [30]:
# Splitting by unique values of the classes.
df = df[ds_cols+[target_col]]
df[target_col] = df[target_col].astype(str)
df_fraud = df.query('Class == "1"').reset_index(drop=True).copy()
df_truth = df.query('Class == "0"').reset_index(drop=True).copy()

Results = []
    
# Each seed is a experiment.
for seed in split_seeds:

    # Start the experiment
    df_fraud_train, df_fraud_test = train_test_split(df_fraud, test_size=N_fraud_test, random_state=seed)
    df_truth_train, df_truth_test = train_test_split(df_truth, train_size=N_truth_train, test_size=N_truth_test, random_state=seed)
    df_train = pd.concat([df_fraud_train, df_truth_train]).reset_index(drop=True)
    df_test = pd.concat([df_fraud_test, df_truth_test]).reset_index(drop=True)
    del df_fraud_test, df_truth_test, df_fraud_train, df_truth_train
    
    # Training step
    cls = RandomForestMC(
        n_trees=T, target_col='Class', max_discard_trees=D,
        delta_th=delta_th, th_start=th_start,
        batch_train_pclass=N_T, batch_val_pclass=N_V,
        th_decease_verbose = False, get_best_tree=get_best_tree
    )
    cls.process_dataset(df_train)
    cls.fitParallel(max_workers=16, thread_parallel_method=False)
    y_pred = cls.testForest(df_test)
    y_test = df_test[target_col].tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, average='binary')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    
    Results.append((seed, 'RFMC', F1_M, AUC_ROC_M, RECALL, TP_1))
    
    MLPC = MLPClassifier(hidden_layer_sizes=(200,), max_iter=10000)
    MLPC.fit(X_train_under, y_train_under)
    y_pred = MLPC.predict(X_test)
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, average='binary')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    
    Results.append((seed, 'MLPC', F1_M, AUC_ROC_M, RECALL, TP_1))

Planting the forest:   0%|          | 0/34 [00:00<?, ?it/s]

Process ForkProcess-16:
Process ForkProcess-14:
Process ForkProcess-8:
Process ForkProcess-9:
Process ForkProcess-4:
Process ForkProcess-3:
Process ForkProcess-13:
Process ForkProcess-11:
Process ForkProcess-7:
Process ForkProcess-1:
Process ForkProcess-15:
Process ForkProcess-6:
Process ForkProcess-5:
Process ForkProcess-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/multiprocessing/p

KeyboardInterrupt: 

  File "/usr/local/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/local/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/local/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/local/lib/python3.9/multiprocessing/queues.py", line 102, in get
    with self._rlock:
Process ForkProcess-2:
Process ForkProcess-12:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/